<a href="https://colab.research.google.com/github/MiraHatoum/-AAI614_Hatoum/blob/main/datacleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas


In [3]:
from google.colab import files

# Upload the train.tsv file
uploaded = files.upload()


Saving train.tsv.7z to train.tsv.7z


In [5]:
!apt-get install p7zip-full

# Extract the .7z file
!7z e train.tsv.7z
import pandas as pd

# Load the extracted TSV file into a DataFrame
df = pd.read_csv("train.tsv", sep="\t")

# Display the first few rows
df.head()



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 77912192 bytes (75 MiB)

Extracting archive: train.tsv.7z
--
Path = train.tsv.7z
Type = 7z
Physical Size = 77912192
Headers Size = 122
Method = LZMA2:24
Solid = -
Blocks = 1

  0%      3% - train.tsv                  7% - train.tsv                 11% - train.tsv                 14% - train.tsv                 18% - train.tsv                

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity


In [6]:
# Count rows with blank 'category_name'
blank_category_rows = df['category_name'].isnull().sum()
print(f"Number of rows with blank 'category_name': {blank_category_rows}")

# Remove rows with blank 'category_name'
df = df.dropna(subset=['category_name']).reset_index(drop=True)


Number of rows with blank 'category_name': 6327


In [13]:
# Use a sample of unique names
unique_names_sample = df['name'].dropna().sample(n=1000, random_state=42).unique()

# Run fuzzy matching on the sample
clusters = {}
for name in unique_names_sample:
    matches = process.extract(name, unique_names_sample, limit=5, scorer=process.fuzz.token_sort_ratio)
    clusters[name] = [match[0] for match in matches if match[1] > 80]

# Number of clusters in the sample
print(f"Number of clusters found in the sample: {len(clusters)}")



Number of clusters found in the sample: 999


In [9]:
# Split 'category_name' into multiple columns
category_columns = df['category_name'].str.split('/', expand=True)

# Dynamically generate column names based on the number of splits
category_columns.columns = [f"Category_{i+1}" for i in range(category_columns.shape[1])]

# Concatenate the new columns with the original DataFrame
df = pd.concat([df, category_columns], axis=1)

# Display the first few rows of the updated DataFrame
df.head()

# Count the number of new columns created
print(f"Number of new columns created: {category_columns.shape[1]}")


Number of new columns created: 5


In [15]:
# Remove original 'Category_X' columns
df = df.drop(columns=[col for col in df.columns if col.startswith('Category_')])

# Add only non-empty new category columns
df = pd.concat([df, non_empty_columns], axis=1)

# Display the first few rows
df.head()


,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,Category_1,Category_2,Category_3,Category_4,Category_5
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet,Men,Tops,T-shirts,None,None
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...,Electronics,Computers & Tablets,Components & Parts,None,None
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...,Women,Tops & Blouses,Blouse,None,None
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...,Home,Home Décor,Home Décor Accents,None,None
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity,Women,Jewelry,Necklaces,None,None


In [16]:
# Create 'high_priced' column
df['high_priced'] = df['price'].apply(lambda x: 1 if x > 100 else 0)

# GREL Equivalent:
# if(value > 100, 1, 0)


In [17]:
# Print the first few rows of just the 'high_priced' column
print(df['high_priced'].head())


0    0
1    0
2    0
3    0
4    0
Name: high_priced, dtype: int64


In [18]:
# Replace null values in 'brand_name'
df['brand_name'] = df['brand_name'].fillna('Unbranded')

# GREL Equivalent:
# if(isBlank(value), "Unbranded", value)


In [19]:
# Create 'high_priced' column
df['high_priced'] = df['price'].apply(lambda x: 1 if x > 100 else 0)

# GREL Equivalent:
# if(value > 100, 1, 0)


In [20]:
# Print the first few rows of just the 'high_priced' column
print(df['high_priced'].head())

0    0
1    0
2    0
3    0
4    0
Name: high_priced, dtype: int64


In [21]:
# Create 'has_offer' column
df['has_offer'] = df['item_description'].apply(
    lambda x: 1 if any(keyword in str(x).lower() for keyword in ['discount', 'offer', 'sale']) else 0
)

# GREL Equivalent:
# if(or(contains(value, "discount"), contains(value, "offer"), contains(value, "sale")), 1, 0)


In [23]:
# Display the first 6 rows of 'price' and 'high_priced' columns
print(df[['price', 'high_priced']].head(10))


   price  high_priced
0   10.0            0
1   52.0            0
2   10.0            0
3   35.0            0
4   44.0            0
5   59.0            0
6   64.0            0
7    6.0            0
8   19.0            0
9    8.0            0
